In [1]:
%pwd

'e:\\DeepLearning\\Kidney-Disease-Classification\\research'

In [2]:
import os
import sys
from pathlib import Path

import os
from pathlib import Path

# Set your working directory explicitly
project_dir = Path(r"E:\DeepLearning\Kidney-Disease-Classification")
os.chdir(project_dir)
print("Working directory set to:", os.getcwd())


Working directory set to: E:\DeepLearning\Kidney-Disease-Classification


In [3]:
import tensorflow as tf
from pathlib import Path

# Enable eager execution (must come before model loading/training)
tf.config.run_functions_eagerly(True)
print("Eager Execution Enabled:", tf.executing_eagerly())



Eager Execution Enabled: True


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf


In [6]:
from pathlib import Path

project_dir = Path(r"E:\DeepLearning\Kidney-Disease-Classification")
ROOT_DIR = Path.cwd()
CONFIG_FILE_PATH = ROOT_DIR / "config" / "config.yaml"
PARAMS_FILE_PATH = ROOT_DIR / "params.yaml"

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories
import tensorflow as tf

In [8]:
# NOTE: Please ensure that the cell defining TrainingConfig (cell 3) has been executed before running this cell.

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        return training_config


In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [10]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path,
            compile=False
        )
        self.model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='categorical_crossentropy',  # or 'binary_crossentropy' if 2 classes
        metrics=['accuracy']
    )
    

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [13]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-08-02 09:38:20,888: INFO: common: yaml file: E:\DeepLearning\Kidney-Disease-Classification\config\config.yaml loaded successfully]
[2025-08-02 09:38:20,891: INFO: common: yaml file: E:\DeepLearning\Kidney-Disease-Classification\params.yaml loaded successfully]
[2025-08-02 09:38:20,892: INFO: common: created directory at: artifacts]
[2025-08-02 09:38:20,895: INFO: common: created directory at: artifacts\training]
Found 151 images belonging to 2 classes.
Found 608 images belonging to 2 classes.


e:\DeepLearning\Kidney-Disease-Classification\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
e:\DeepLearning\Kidney-Disease-Classification\.venv\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


38/38 ━━━━━━━━━━━━━━━━━━━━ 236s 6s/step - accuracy: 0.8734 - loss: 0.2986 - val_accuracy: 0.8264 - val_loss: 0.3212
[2025-08-02 09:42:18,706: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


In [12]:
%pwd

'E:\\DeepLearning\\Kidney-Disease-Classification'